In [59]:
%reset
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import model_selection, preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [61]:
train_set = pd.read_csv('/Users/gayathrydasika/Documents/Data_science/DATASETS/Mercedez/train.csv')
test_set = pd.read_csv('/Users/gayathrydasika/Documents/Data_science/DATASETS/Mercedez/test.csv')

train_col_names = train_set.columns
fac_names=train_col_names[2:10]
train_set.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [62]:
#Conversion using label encoder
clean_df = train_set.copy()

for f in clean_df.columns:
    if clean_df[f].dtype == 'object':
        label = preprocessing.LabelEncoder()
        label.fit(list(clean_df[f].values))
        clean_df[f] = label.transform(list(clean_df[f].values))
clean_df.head()


,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,32,23,17,0,3,24,9,14,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,32,21,19,4,3,28,11,14,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,20,24,34,2,3,27,9,23,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,20,21,34,5,3,27,11,4,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,20,23,34,5,3,12,3,13,...,0,0,0,0,0,0,0,0,0,0


In [63]:
train_y = clean_df.y.values
train_x = clean_df.drop(["y"],axis=1)
train_x = train_x.drop(["ID"],axis=1)
train_x = train_x.values

In [68]:
#Fitting XGB regressor 
model = xgb.XGBRegressor()
model.fit(train_x,train_y)
print model





XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)


In [74]:
id_vals = test_set.ID.values

clean_test = test_set.copy()
for f in clean_test.columns:
    if clean_test[f].dtype == 'object':
        label = preprocessing.LabelEncoder()
        label.fit(list(clean_test[f].values))
        clean_test[f] = label.transform(list(clean_test[f].values))
clean_test.fillna((-999), inplace=True)
test = clean_test.drop(['ID'],axis=1)
x_test = test.values

#testset is ready

array([[21, 23, 34, ...,  0,  0,  0],
       [42,  3,  8, ...,  0,  0,  0],
       [21, 23, 17, ...,  0,  0,  0],
       ..., 
       [47, 23, 17, ...,  0,  0,  0],
       [ 7, 23, 17, ...,  0,  0,  0],
       [42,  1,  8, ...,  0,  0,  0]])

In [78]:
#Predict 
output = model.predict(data=x_test)
final_df = pd.DataFrame()
final_df["ID"] = id_vals
final_df["Prediction"] = output
final_df.to_csv("Output_1.csv")

In [11]:
xgb_params = {
    
    'eta': 0.05,
    'max_depth': 8,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'auc',
    'silent': 1      
}

In [12]:
dtrain = xgb.DMatrix(train_x, train_y, feature_names=train_x.columns.values) #Data Matrix for XGB
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=100) #model created

In [53]:
fig, ax = plt.subplots(figsize=(20,50))
xgb.plot_importance(model, height=0.5, ax=ax)
plt.savefig('Feature.png')

In [25]:
test_sheet = pd.read_csv('test.csv')
test_id = test_sheet.ID.values
test_sheet.head()
train_x.head()

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,32,23,17,0,3,24,9,14,0,0,...,0,0,1,0,0,0,0,0,0,0
1,32,21,19,4,3,28,11,14,0,0,...,1,0,0,0,0,0,0,0,0,0
2,20,24,34,2,3,27,9,23,0,0,...,0,0,0,0,0,0,1,0,0,0
3,20,21,34,5,3,27,11,4,0,0,...,0,0,0,0,0,0,0,0,0,0
4,20,23,34,5,3,12,3,13,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
test_data = test_sheet.drop(["ID"],axis=1)
test_data.head()

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,az,v,n,f,d,t,a,w,0,0,...,0,0,0,1,0,0,0,0,0,0
1,t,b,ai,a,d,b,g,y,0,0,...,0,0,1,0,0,0,0,0,0,0
2,az,v,as,f,d,a,j,j,0,0,...,0,0,0,1,0,0,0,0,0,0
3,az,l,n,f,d,z,l,n,0,0,...,0,0,0,1,0,0,0,0,0,0
4,w,s,as,c,d,y,i,m,0,0,...,1,0,0,0,0,0,0,0,0,0


In [48]:
clean_test = test_data.copy()

for f in clean_test.columns:
    if clean_test[f].dtype == 'object':
        label = preprocessing.LabelEncoder()
        label.fit(list(clean_test[f].values))
        clean_test[f] = label.transform(list(clean_test[f].values))
clean_test.head()

clean_test.fillna((-999), inplace=True)

clean_test=np.array(clean_test) 

clean_test = clean_test.astype(float)

In [44]:
m_predict = model.copy()

In [49]:
dtrain = xgb.DMatrix(train_x, train_y, feature_names=train_x.columns.values) #Data Matrix for XGB
model = xgb.train(dict(xgb_params, silent=0), dtrain=dtrain, num_boost_round=100)#model created
model.predict(data=clean_tes)

AttributeError: 'numpy.ndarray' object has no attribute 'feature_names'